# import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# Load data (gantilah dengan path filemu)
df_job = pd.read_csv('scrape glints companies and jobs/scraping/job_posti_with_details.csv')
df_company = pd.read_excel(r"scrape glints companies and jobs/scraping/company_df.xlsx")

# Lihat overview awal
print(df_job.info())
print(df_company.info())

In [ ]:
# df_job.head()

# data preprocessing

In [ ]:
# Convert kolom Timestamp_Scraped ke datetime
df_job['Timestamp_Scraped'] = pd.to_datetime(df_job['Timestamp_Scraped'], errors='coerce')

# Hapus spasi berlebih dari semua kolom object
for col in df_job.select_dtypes(include='object'):
    df_job[col] = df_job[col].astype(str).str.strip()

for col in df_company.select_dtypes(include='object'):
    df_company[col] = df_company[col].astype(str).str.strip()


In [ ]:
# Convert kolom Timestamp_Scraped ke datetime
df_job['Timestamp_Scraped'] = pd.to_datetime(df_job['Timestamp_Scraped'], errors='coerce')

# Hapus spasi berlebih dari semua kolom object
for col in df_job.select_dtypes(include='object'):
    df_job[col] = df_job[col].astype(str).str.strip()

for col in df_company.select_dtypes(include='object'):
    df_company[col] = df_company[col].astype(str).str.strip()


In [ ]:
# Convert kolom Timestamp_Scraped ke datetime
df_job['Timestamp_Scraped'] = pd.to_datetime(df_job['Timestamp_Scraped'], errors='coerce')

# Hapus spasi berlebih dari semua kolom object
for col in df_job.select_dtypes(include='object'):
    df_job[col] = df_job[col].astype(str).str.strip()

for col in df_company.select_dtypes(include='object'):
    df_company[col] = df_company[col].astype(str).str.strip()


In [ ]:
df_company_email = df_company[df_company['Cleaned_Email'].notna()].copy()

In [ ]:
df_merged = pd.merge(df_job, df_company_email, left_on='Company', right_on='Name', how='inner')

In [ ]:
import re

def extract_days(text):
    if 'hari' in text:
        match = re.search(r'(\d+)', text)
        return int(match.group()) if match else np.nan
    elif 'bulan' in text:
        match = re.search(r'(\d+)', text)
        return int(match.group()) * 30 if match else np.nan
    return np.nan

df_merged['Days_Posted'] = df_merged['Job Posted'].apply(extract_days)


In [ ]:
df_merged['Email Domain'] = df_merged['Cleaned_Email'].str.extract(r'@([\w\.-]+)')


In [ ]:
import matplotlib.ticker as ticker

sns.set(style='whitegrid')
plt.figure(figsize=(12, 6))


In [ ]:
# 2. Mengecek apakah kolom 'Job Link' punya duplikat
jumlah_duplikat_joblink = df_merged['Job Link'].duplicated().sum()
print(f"Jumlah duplikat dalam kolom 'Job Link': {jumlah_duplikat_joblink}")

# Jika mau lihat link-linknya
duplikat_joblink = df_merged[df_merged['Job Link'].duplicated(keep=False)].sort_values('Job Link')
print("Data yang memiliki duplikat Job Link:\n", duplikat_joblink)

In [ ]:
# Hapus duplikat berdasarkan kolom Job Link
df_merged = df_merged.drop_duplicates(subset='Job Link', keep='first')

# Cek jumlah baris setelah deduplikasi
print("Jumlah baris setelah hapus duplikat Job Link:", len(df_merged))


# data exploration

In [ ]:
top_companies = df_merged['Company'].value_counts().nlargest(15)
top_companies.plot(kind='barh', color='skyblue')
plt.title("15 Perusahaan dengan Jumlah Lowongan Terbanyak")
plt.xlabel("Jumlah Lowongan")
plt.gca().invert_yaxis()
plt.show()


In [ ]:
top_companies = df_merged['Company'].value_counts().nlargest(15)
top_companies.plot(kind='barh', color='skyblue')
plt.title("15 Perusahaan dengan Jumlah Lowongan Terbanyak")
plt.xlabel("Jumlah Lowongan")
plt.gca().invert_yaxis()
plt.show()


In [ ]:
df_merged['Industry'].value_counts().nlargest(10).plot(kind='bar', color='lightgreen')
plt.title("10 Industri Teratas Berdasarkan Jumlah Lowongan")
plt.ylabel("Jumlah Lowongan")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(df_merged['Days_Posted'].dropna(), bins=20, kde=True, color='salmon')
plt.title("Distribusi Lama Posting Lowongan")
plt.xlabel("Hari")
plt.show()

print("Rata-rata Durasi:", df_merged['Days_Posted'].mean())
print("Median Durasi:", df_merged['Days_Posted'].median())


In [ ]:
df_merged['Min Education'].value_counts().plot(kind='barh', color='orange')
plt.title("Distribusi Jenjang Pendidikan Minimum")
plt.xlabel("Jumlah Lowongan")
plt.gca().invert_yaxis()
plt.show()


In [ ]:
df_merged['Employment Type'].value_counts().plot(kind='pie', autopct='%1.1f%%', startangle=140, colors=sns.color_palette("pastel"))
plt.ylabel('')
plt.title("Distribusi Tipe Employment")
plt.show()


In [ ]:
from collections import Counter

skills_list = df_merged['Skills'].dropna().str.split(', ')
all_skills = [skill for sublist in skills_list for skill in sublist]
common_skills = Counter(all_skills).most_common(15)

skills_df = pd.DataFrame(common_skills, columns=['Skill', 'Count'])

plt.figure(figsize=(10, 6))
sns.barplot(data=skills_df, y='Skill', x='Count', palette='coolwarm')
plt.title("15 Skills yang Paling Sering Dicari")
plt.show()


In [ ]:
df_merged['Email Domain'].value_counts().head(10).plot(kind='bar', color='purple')
plt.title("10 Domain Email Perusahaan Terpopuler")
plt.ylabel("Jumlah Perusahaan")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
from collections import Counter

# Gabung semua skill jadi satu list
skills_list = df_merged['Skills'].dropna().str.split(',').sum()
top_skills = Counter([skill.strip() for skill in skills_list if skill.strip() != ""]).most_common(10)

# Visualisasi
skills, counts = zip(*top_skills)
plt.figure(figsize=(10,5))
sns.barplot(x=counts, y=skills)
plt.title('Top 10 Skills yang Paling Dibutuhkan')
plt.xlabel('Jumlah')
plt.tight_layout()
plt.show()


In [ ]:
df_merged.head(-5)

In [ ]:
df_merged.columns

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df_merged.info()

In [ ]:
df_merged = df_merged.rename(columns={
    'Location_x': 'Job_Location',
    'Location_y': 'Company_Location',
    'Company': 'Company_Name'  # untuk menghindari konflik keyword 'Company'
})


In [ ]:
df_merged['Skill_Count'] = df_merged['Skills'].apply(lambda x: len(x.split(',')) if pd.notna(x) else 0)


In [ ]:
# Provinsi: elemen terakhir
df_merged['Job_Province'] = df_merged['Job_Location'].apply(
    lambda x: x.split(',')[-1].strip() if pd.notna(x) else np.nan
)

# Kota/Kabupaten: elemen ke-2
df_merged['Job_City'] = df_merged['Job_Location'].apply(
    lambda x: x.split(',')[1].strip() if pd.notna(x) and len(x.split(',')) > 1 else np.nan
)


In [ ]:
import re

def extract_salary_min(s):
    try:
        s_clean = s.replace('.', '').replace(',', '').upper()
        result = re.findall(r'IDR(\d+)', s_clean)
        return int(result[0]) if result else np.nan
    except:
        return np.nan

def extract_salary_max(s):
    try:
        s_clean = s.replace('.', '').replace(',', '').upper()
        result = re.findall(r'IDR\d+\s*-\s*(\d+)', s_clean)
        return int(result[0]) if result else np.nan
    except:
        return np.nan

df_merged['Salary_Min'] = df_merged['Salary Range'].apply(extract_salary_min)
df_merged['Salary_Max'] = df_merged['Salary Range'].apply(extract_salary_max)


In [ ]:
# df_merged.to_csv("final_dataset_jobs.csv", index=False)


# data cleaning

In [ ]:
df_merged.info()

In [ ]:
df_merged.head()

## edit education

In [ ]:
df_merged['Min Education'].value_counts(dropna=False).reset_index().rename(columns={'index':'Min Education','Min Education':'Count'})


In [ ]:
edu_mapping = {
    "Minimal SMA/SMK": "SMA/SMK",
    "Minimal Sarjana (S1)": "S1",
    "Minimal Diploma (D1 - D4)": "D1-D4",
    "Minimum Bachelor’s Degree": "S1",
    "Minimal SD": "SD",
    "Minimal SMP": "SMP",
    "Minimum College Degree": "D1-D4",
    "Minimal Magister (S2)": "S2",
    "Minimum Associate Degree": "D1-D4",
    "Minimum Senior/Vocational High School": "SMA/SMK"
}


In [ ]:
df_merged['Min Education'] = df_merged['Min Education'].replace(edu_mapping)
df_merged['Min Education'] = df_merged['Min Education'].fillna('Unknown')


In [ ]:
df_merged.isna().sum().reset_index().rename(columns={'index': 'Column', 0: 'Missing Values'})


In [ ]:
df_merged[['Salary Range', 'Salary_Min', 'Salary_Max']].sample(10)


In [ ]:
df_merged.to_excel("new_full_dataset_jobs.xlsx", index=False)


In [ ]:
# # Pastikan kolom Cleaned_Email dibaca sebagai string
# df_merged['Cleaned_Email'] = df_merged['Cleaned_Email'].astype(str)

# # Bersihkan spasi dan string 'nan' yang sebenarnya bukan NaN
# df_merged['Cleaned_Email_clean'] = df_merged['Cleaned_Email'].str.strip().replace({'': np.nan, 'nan': np.nan})

# # Filter baris yang benar-benar punya email valid
# filtered_df = df_merged[df_merged['Cleaned_Email_clean'].notna()]

# # Simpan ke file Excel
# filtered_df.to_excel("01.filter_final_dataset_jobs.xlsx", index=False)


In [ ]:
import numpy as np
import pandas as pd

# Pastikan kolom Cleaned_Email dan Phone Number bertipe string
df_merged['Cleaned_Email'] = df_merged['Cleaned_Email'].astype(str)
df_merged['Phone Number'] = df_merged['Phone Number'].astype(str)

# Bersihkan spasi dan ganti 'nan' menjadi NaN
df_merged['Cleaned_Email_clean'] = df_merged['Cleaned_Email'].str.strip().replace(
    {'': np.nan, 'nan': np.nan}
)
df_merged['Phone_Number_clean'] = df_merged['Phone Number'].str.strip().replace(
    {'': np.nan, 'nan': np.nan}
)

# Filter: setidaknya salah satu (email atau phone) harus ada
filtered_df = df_merged[
    df_merged['Cleaned_Email_clean'].notna() | df_merged['Phone_Number_clean'].notna()
]

# Simpan ke file Excel
filtered_df.to_excel("00.new_filter_final_dataset_jobs.xlsx", index=False)

print(f"✅ Total data sebelum filter: {len(df_merged)}, setelah filter: {len(filtered_df)}")


In [ ]:
filtered_df.info()

In [ ]:
# # Pastikan kolom Cleaned_Email dibaca sebagai string
# df_company['Cleaned_Email'] = df_company['Cleaned_Email'].astype(str)

# # Bersihkan spasi dan string 'nan' literal
# df_company['Cleaned_Email_clean'] = df_company['Cleaned_Email'].str.strip().replace({'': np.nan, 'nan': np.nan})

# # Filter hanya baris dengan email yang valid
# filtered_df = df_company[df_company['Cleaned_Email_clean'].notna()]

# # Simpan hasil ke file Excel baru
# filtered_df.to_excel("01.df_company.xlsx", index=False)

## visualisasi score card

In [ ]:
import numpy as np
import pandas as pd

# Total perusahaan unik
num_unique_companies = filtered_df['Full URL'].nunique()

# Total job unik
num_unique_jobs = filtered_df['Job Link'].nunique()

# Jumlah data per provinsi
jobs_per_province = filtered_df['Job_Province'].value_counts(dropna=False)

# Rata-rata hari tayang
avg_days_posted = filtered_df['Days_Posted'].mean()

# Rata-rata skill per job
avg_skills = filtered_df['Skill_Count'].mean()

# Jumlah baris yang punya email valid
num_with_email = filtered_df['Cleaned_Email_clean'].notna().sum()

# Jumlah baris yang punya phone valid
num_with_phone = filtered_df['Phone_Number_clean'].notna().sum()

# Buat summary dalam bentuk dictionary
summary = {
    "Total Data Filtered": len(filtered_df),
    "Unique Companies (Full URL)": num_unique_companies,
    "Unique Job Links": num_unique_jobs,
    "Average Days Posted": round(avg_days_posted, 2),
    "Average Skills per Job": round(avg_skills, 2),
    "Has Valid Email": num_with_email,
    "Has Valid Phone Number": num_with_phone,
}

# Konversi ke DataFrame untuk tampilan tabel
scorecard_df = pd.DataFrame(list(summary.items()), columns=["Metric", "Value"])

# Tampilkan scorecard
print(scorecard_df)

# Jika mau lihat jobs per provinsi lebih detail:
print("\nJobs per province:")
print(jobs_per_province)


# explode skill for visualisasi

In [ ]:
df_skills = filtered_df.assign(Skills=filtered_df['Skills'].str.split(',')).explode('Skills')

# Bersihkan spasi
df_skills['Skills'] = df_skills['Skills'].str.strip()

df_skills.head()

In [ ]:
df_skills.to_excel("00.skilss_explode_df.xlsx", index=False)